In [ ]:
import pandas as pd
import numpy as np

In [ ]:
token = "token"
!git clone https://{token}@github.com/Raghav-Sahni/Hope_speech.git

Cloning into 'Hope_speech'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 700 (delta 151), reused 228 (delta 125), pack-reused 439
Receiving objects: 100% (700/700), 68.67 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (411/411), done.


In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
eng_df = pd.read_csv("/content/Hope_speech/Data/PreprocessedData/english_train_preprocess.csv")

In [ ]:
eng_df.head()

,Unnamed: 0,text,label,preprocessed_text
0,0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech,these tiktoks radiate gay chaotic energy and i...
1,1,@Champions Again He got killed for using false...,Non_hope_speech,<user> again he got killed for using false money
2,2,It's not that all lives don't matter,Non_hope_speech,its not that all lives dont matter
3,3,Is it really that difficult to understand? Bla...,Non_hope_speech,is it really that difficult to understand blac...
4,4,Whenever we say black isn't that racists? Why...,Non_hope_speech,whenever we say black isnt that racists why do...


In [ ]:
aug_df = pd.read_csv("/content/Hope_speech/Data/AugmentedData/english_train_augmented.csv")

In [ ]:
aug_df.head()

,Unnamed: 0,augmented_text,label
0,0,engineer network here 23 presently working as ...,Hope_speech
1,1,engineer network here 23 and presently instruc...,Hope_speech
2,2,Still im hiding grammatical gender to my paren...,Hope_speech
3,3,im hide still my gender to my parents and they...,Hope_speech
4,4,all subject without lives that we never have p...,Hope_speech


In [ ]:
sentences = aug_df["augmented_text"].values

In [ ]:
sentences = [i.split() for i in sentences]

In [ ]:
w2v = Word2Vec(sentences = sentences, window = 5)

In [ ]:
print(sentences[1])

['engineer', 'network', 'here', '23', 'and', 'presently', 'instructor', 'teaching', 'and', 'men', 'women', 'await', 'to', 'be', 'in', 'information', 'technology', '=', 'adjacent', 'i', 'need', 'to', 'at', 'teach', 'a', 'university']


In [ ]:
embeddings_w2v_aug = []
for i in sentences:
  sen = []
  for j in i:
    try:
      sen.append(w2v.__getitem__(i))
    except:
      sen.append(0)
  embeddings_w2v_aug.append(np.mean(sen))

<ipython-input-64-1954f22272f9>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  sen.append(w2v.__getitem__(i))


In [ ]:
len(embeddings_w2v_aug)

42382

In [ ]:
import pickle

with open('gensim_custom_w2v_embeddings_augmented_train.pickle', 'wb') as handle:
    pickle.dump(embeddings_w2v_aug, handle, protocol=pickle.HIGHEST_PROTOCOL)